In [111]:
!pip install --upgrade pip
!pip cache purge
!pip install opencv-python
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install scipy

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/c9/bc/b7db44f5f39f9d0494071bddae6880eb645970366d0a200022a1a93d57f5/pip-25.0.1-py3-none-any.whl (Caused by ReadTimeoutError("HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out. (read timeout=15)"))

Files removed: 8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 4.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


In [112]:
import os
import shutil
import random

In [113]:
base = 'imagens/Dog'

In [114]:
train_dir = 'dataset/train/not_cats'
val_dir = 'dataset/validation/not_cats'

In [115]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [116]:
all_files = os.listdir(base)

In [117]:
random.shuffle(all_files)

In [118]:
split = int(len(all_files)* 0.8)

In [119]:
train = all_files[:split]
val = all_files[split:]

In [120]:
for img in train:
    shutil.move(os.path.join(base, img), os.path.join(train_dir, img))

for img in val:
    shutil.move(os.path.join(base, img), os.path.join(val_dir, img))

# Treinar o modelo

In [5]:
import tensorflow as tf
import scipy
import keras.preprocessing.image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

2025-03-31 12:35:02.906466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
dataset_path = 'dataset'
img_Size = (24, 24)
batch_size = 128
epochs = 10

In [37]:
teste_data = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True
)

In [24]:
teste_val = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True
)

In [25]:
train_gen = teste_data.flow_from_directory(
    os.path.join(dataset_path, 'train'),
    target_size=img_Size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 19996 images belonging to 2 classes.


In [26]:
val_gen = teste_data.flow_from_directory(
    os.path.join(dataset_path, 'validation'),
    target_size=img_Size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5000 images belonging to 2 classes.


In [27]:
modelo = Sequential([

    Conv2D(32, (3, 3), activation='relu', input_shape=(img_Size[0], img_Size[0], 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', input_shape=(img_Size[0], img_Size[0], 3)),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', input_shape=(img_Size[0], img_Size[0], 3)),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),


    Dropout(0.5),
    Dense(1, activation='sigmoid') # resposta

])

In [28]:
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
epochs = 1

In [36]:
modelo.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs
)

1667/1667 [==============================] - 350s 210ms/step - loss: 0.6822 - accuracy: 0.5524 - val_loss: 0.6536 - val_accuracy: 0.6170


In [38]:
modelo.save('cat_noCat.keras')

1.10.1
